# 📌 Project Overview: Sales & Operations Analytics for an Underwear Wholesaler

Welcome to the end-to-end analytics pipeline designed for a wholesale clothing distributor specializing in intimate apparel. This project simulates the real-world process of extracting insights from a raw operational dataset by applying cloud-based data engineering and analytics techniques using **Apache Spark** and **Databricks**.

Through a series of structured steps — from data ingestion and quality checks to schema modeling and business intelligence queries — building a solid analytical foundation that supports decision-making across departments like product, sales, customer success, operations, and logistics.

---

## 🎯 Objective

The primary objective of this project is to **build a robust and scalable data pipeline** capable of:

- Modeling a retail business using a **star schema**;
- Enriching and cleaning raw datasets into **analysis-ready structures**;
- **Answering business-critical questions** using SQL-based queries on Spark tables;
- Supporting insights generation with a focus on **revenue optimization**, **customer segmentation**, and **sales performance**.

---

## ❓ Key Business Questions Answered

This project explores a variety of business-critical topics, including:

- What are the company’s top revenue-generating **products**, **categories**, and **regions**?
- Which **customer segments** and **acquisition channels** drive the most revenue?
- How does **employee performance** vary and which reps generate the most value?
- What is the **geographic distribution** of sales and which cities offer growth opportunities?
- How does **customer loyalty** contribute to sustained revenue?
- What can we learn from **shipping patterns**, **payment coverage**, and **churn behavior**?

Each of these questions is addressed through clearly defined queries and visual breakdowns throughout the notebook.

---

## 🗂️ Project Structure Overview

Below is a complete breakdown of the sections and subsections covered throughout this project:

---

### 0 – 🧰 Project Initialization
- **0.1 – 🖥️ Environment Setup**
- **0.2 – 🌐 Accessing and Listing Remote Files from GitHub**

---

### 1 – ⚙️ Data Loading & Initial Fixes
- **1.0 – 💾 Downloading and Loading CSV Files into Spark**
- **1.1 – ⚠️ Fixing Encoding Issues in customers_df**
- **1.2 – ⚠️ Fixing Encoding Issues in employees_df**

---

### 2 – 🧹 Initial Data Quality Checks
- **2.1 – 🔍 Quick Data Quality Audit (Nulls, Duplicates & Uniqueness)**
- **2.2 – 🧼 Null Value Analysis & Data Cleaning Decisions**

---

### 3 – 🧱 Building the Enriched Fact Table
- **3.1 – 📦 Create Base Fact Table (Orders + Order Details)**
- **3.2 – 👥 Enrich Fact Table with Customer and Employee Information**
- **3.3 – 🚚 Enrich Fact Table with Product and Shipping Information**
- **3.4 – 💳 Enrich Fact Table with Payment Information**

---

### 4 – ⭐ Building the Star Schema: Dimensions & Fact Table
- **4.1 – 🧑‍💼 Create `dim_customers`: Customer Dimension Table**
- **4.2 – 🛍️ Create `dim_products`: Product Dimension Table**
- **4.3 – 👤 Create `dim_employees`: Employee Dimension Table**
- **4.4 – 🚚 Create `dim_shipping_methods`: Shipping Method Dimension**
- **4.5 – 💳 Create `dim_payments`: Payment Dimension Table**
- **4.6 – 📅 Create `dim_date`: Date Dimension Table**
- **4.7 – 🧾 Create Final `fact_sales` Table**
- ⭐ **Star Schema Completed**

---

### 5 – 📊 Business Questions & Strategic Insights
- **5.0 – 🧠 Registering Temp Views for SQL Queries**
- **5.1 – 🧾 Sales Orders vs. Paid Orders**
- **5.2 – 🧍 Revenue by Product Gender**
- **5.3 – 🩲 Revenue by Product Category**
- **5.4 – 🌍 Regional Revenue Breakdown — Top 10 Cities by Total Revenue**
- **5.5 – 🔍 Revenue by Lead Source**
- **5.6 – 🧩 Revenue by Customer Class**
- **5.7 – 🔁 Revenue by Customer Status (Active vs. Discontinued)**
- **5.8 – 🏆 Top 10 Customers – Performance Deep Dive**
- **5.9 – 🧑‍💼 Employee Performance Analysis**
- **5.10 – 📆 Revenue Trend Over Time**
- **5.11 – 💰 Average Order Value (AOV) by Customer Class**
- **5.12 – 🔁 Revenue by Customer Loyalty**
- **5.13 – 🚚 Shipping Method Overview**

---

### 6 – ✅ Final Considerations
- Summary of findings, data quality remarks, business takeaways, and suggestions for future work.

## 0 – 🧰 Project Initialization

This section sets the stage for the entire project by preparing the technical environment and retrieving references to the remote data.

### Subsections:

- **0.1 – 💻 Environment Setup**  
- **0.2 – 🌐 Accessing and Listing Remote Files**  

### 0.1 - 💻 Environment Setup

This section prepares the project environment by importing necessary libraries and initializing the Spark session. These setup steps enable interaction with the Databricks file system, HTTP requests to GitHub, and large-scale data processing with PySpark.

In [0]:
# Core Libraries
import requests

# Databricks Utilities
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, dayofmonth, dayofweek, weekofyear, quarter, date_format, trim, regexp_replace

# Create Spark Session (if not already created)
spark = SparkSession.builder.appName("GitHub_CSV_Loading").getOrCreate()

# File System Utilities
import os
import shutil


###0.2 – 🌐 Accessing and Listing Remote Files from GitHub

After preparing the environment, this step uses the GitHub API to dynamically retrieve the list of CSV files to be used in the project. This avoids hardcoding file paths and ensures a scalable and automated ingestion process.

In [0]:
# GitHub API URL for the folder
GITHUB_API_URL = "https://api.github.com/repos/Barbosa6Andre/underwearWholesaling/contents/Underwear%20wholesaler"

# Send a GET request to GitHub API
response = requests.get(GITHUB_API_URL)

# Check if request was successful
if response.status_code == 200:
    files = response.json()
    # Extract file names and raw download URLs
    file_list = [{"name": file["name"], "url": file["download_url"]} for file in files if "download_url" in file]
    
    # Print the file list
    for file in file_list:
        print(f"File: {file['name']} - URL: {file['url']}")
else:
    print(f"Error: Unable to fetch files (Status Code: {response.status_code})")


File: customers.csv - URL: https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/customers.csv
File: employees.csv - URL: https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/employees.csv
File: inventory_transactions.csv - URL: https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/inventory_transactions.csv
File: order_details.csv - URL: https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/order_details.csv
File: orders.csv - URL: https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/orders.csv
File: payment_methods.csv - URL: https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/payment_methods.csv
File: payments.csv - URL: https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/payments.

In [0]:
#check if all files are in the list
print(file_list)

[{'name': 'customers.csv', 'url': 'https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/customers.csv'}, {'name': 'employees.csv', 'url': 'https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/employees.csv'}, {'name': 'inventory_transactions.csv', 'url': 'https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/inventory_transactions.csv'}, {'name': 'order_details.csv', 'url': 'https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/order_details.csv'}, {'name': 'orders.csv', 'url': 'https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/orders.csv'}, {'name': 'payment_methods.csv', 'url': 'https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/payment_methods.csv'}, {'name': 'payments.csv', 'url': 'https://raw.githubusercontent.com/Barb

## 1 – ⚙️ Data Loading & Initial Fixes

This section focuses on the first critical step of the pipeline: **retrieving, loading, and validating the raw CSV files** that will form the foundation of our star schema.

### Subsections:

- **1.0 – 💾 Downloading and Loading CSV Files into Spark**  
- **1.1 – ⚠️ Fixing Encoding Issues in `customers_df`** 
- **1.2 – ⚠️ Fixing Encoding Issues in `employees_df`**  

These steps ensure that all base DataFrames are accessible, readable, and ready for early quality checks.

### 1.0 - 💾 Downloading and Loading CSV Files into Spark

This step handles the full data ingestion process:

- Creates a directory in the Databricks File System (DBFS) to store CSV files.
- Downloads the CSV files from the GitHub repository using their raw URLs.
- Loads each CSV into Spark DataFrames with automatic schema inference.
- Stores all DataFrames in a dictionary and assigns them to individual variables for ease of use.
- Prepares two key lists: `all_dfs` (with DataFrames) and `all_dfs_names` (with their labels) that will be reused throughout the notebook for looping, checking, and tracking.

In [0]:
# Define DBFS and local OS path
folder_name = "tmp/github_csv"
os_path = f"/dbfs/{folder_name}"  # For open()
dbfs_path = f"/{folder_name}"     # For dbutils

# Ensure the folder exists at the OS level
os.makedirs(os_path, exist_ok=True)  # This is the key fix

# Base URL
github_base_url = "https://raw.githubusercontent.com/Barbosa6Andre/underwearWholesaling/main/Underwear%20wholesaler/"

# Dictionary to store DataFrames
dfs = {}

for f in file_list:
    file_name = f["name"]
    file_url = f["url"]
    file_path = os.path.join(os_path, file_name)  # OS-level path to save

    try:
        response = requests.get(file_url)
        response.raise_for_status()

        # Save file to /dbfs/ via local OS-level path
        with open(file_path, "wb") as f:
            f.write(response.content)

        print(f"✅ Saved: {file_name}")

        # Load into Spark DataFrame
        df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"file://{file_path}")
        dfs[file_name] = df

    except Exception as e:
        print(f"❌ Error with {file_name}: {e}")

✅ Saved: customers.csv
✅ Saved: employees.csv
✅ Saved: inventory_transactions.csv
✅ Saved: order_details.csv
✅ Saved: orders.csv
✅ Saved: payment_methods.csv
✅ Saved: payments.csv
✅ Saved: products.csv
✅ Saved: purchase_orders.csv
✅ Saved: shipping_methods.csv
✅ Saved: suppliers.csv


In [0]:
#check if all files were properly saved in the dict
dfs.keys()

Out[5]: dict_keys(['customers.csv', 'employees.csv', 'inventory_transactions.csv', 'order_details.csv', 'orders.csv', 'payment_methods.csv', 'payments.csv', 'products.csv', 'purchase_orders.csv', 'shipping_methods.csv', 'suppliers.csv'])

In [0]:
#Establish each of the saved files as a variable to simplify the code
customers_df = dfs['customers.csv']
employees_df = dfs['employees.csv']
inventory_transactions_df = dfs['inventory_transactions.csv']
order_details_df = dfs['order_details.csv']
orders_df = dfs['orders.csv']
payment_methods_df = dfs['payment_methods.csv']
payments_df = dfs['payments.csv']
products_df = dfs['products.csv']
purchase_orders_df = dfs['purchase_orders.csv']
shipping_methods_df = dfs['shipping_methods.csv']
suppliers_df = dfs['suppliers.csv']

all_dfs = [customers_df, employees_df, inventory_transactions_df, order_details_df, orders_df, payment_methods_df, payments_df, products_df, purchase_orders_df, shipping_methods_df, suppliers_df]
all_dfs_names = ['customers_df', 'employees_df', 'inventory_transactions_df', 'order_details_df', 'orders_df', 'payment_methods_df', 'payments_df', 'products_df', 'purchase_orders_df', 'shipping_methods_df', 'suppliers_df']

### 1.1 - ⚠️ Fixing Encoding Issues in customers_df

The initial import of the `customers.csv` file revealed character encoding issues in the column names. To resolve this, we reload the dataset with UTF-8 encoding and drop malformed rows using Spark's data reading options. We also define a clean schema manually and apply it to ensure that all downstream operations reference valid and readable columns.

In [0]:
customers_df.show(5)

+----------+------------+------+------------------+-------------+--------------------+--------------------+------------+
|CustomerID|CustomerName|Region|           Country|PriceCategory|       CustomerClass|          LeadSource|Discontinued|
+----------+------------+------+------------------+-------------+--------------------+--------------------+------------+
|         1|          C1|Moscow|Russian Federation|            1|Large-Scale Whole...|Referral by the C...|           0|
|         2|          C2|Moscow|Russian Federation|            1|Large-Scale Whole...|Referral by the C...|           0|
|         3|          C3|Moscow|Russian Federation|            1|Large-Scale Whole...|Referral by the C...|           1|
|         4|          C4|Moscow|Russian Federation|            4|Large-Scale Whole...|Referral by the C...|           1|
|         5|          C5|Moscow|Russian Federation|            1|Large-Scale Whole...|Referral by the C...|           0|
+----------+------------+------+

In [0]:
clean_columns_customers = [
    "CustomerID", "CustomerName", "Region", "Country", 
    "PriceCategory", "CustomerClass", "LeadSource", "Discontinued"
]

In [0]:
# Reload raw data with encoding handling
raw_customers_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("encoding", "UTF-8") \
    .option("multiLine", "true") \
    .option("mode", "DROPMALFORMED") \
    .load("file:/dbfs/tmp/github_csv/customers.csv")

In [0]:
customers_df = raw_customers_df.toDF(*clean_columns_customers)
customers_df.printSchema()
customers_df.show(5)

root
 |-- CustomerID: integer (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- PriceCategory: integer (nullable = true)
 |-- CustomerClass: string (nullable = true)
 |-- LeadSource: string (nullable = true)
 |-- Discontinued: integer (nullable = true)

+----------+------------+------+------------------+-------------+--------------------+--------------------+------------+
|CustomerID|CustomerName|Region|           Country|PriceCategory|       CustomerClass|          LeadSource|Discontinued|
+----------+------------+------+------------------+-------------+--------------------+--------------------+------------+
|         1|          C1|Moscow|Russian Federation|            1|Large-Scale Whole...|Referral by the C...|           0|
|         2|          C2|Moscow|Russian Federation|            1|Large-Scale Whole...|Referral by the C...|           0|
|         3|          C3|Moscow|Russian Federat

### 1.2 - ⚠️ Fixing Encoding Issues in employees_df

Similar to `customers_df`, the employee dataset contained improperly encoded characters and malformed values. In this step, we reload the CSV with the appropriate encoding and define a clean schema manually. We also apply string cleaning and type casting functions to ensure IDs are usable integers and names are clean. This prepares the DataFrame for integration into the star schema.

In [0]:
employees_df.show(5)

+--------------------------+-----------------------------+
|��" E m p l o y e e I D " | " E m p l o y e e N a m e " |
+--------------------------+-----------------------------+
|                        1 |                     " E 1 " |
|                        2 |                     " E 2 " |
|                        3 |                     " E 3 " |
|                        5 |                     " E 4 " |
|                        6 |                     " E 5 " |
+--------------------------+-----------------------------+
only showing top 5 rows



In [0]:
clean_columns_employees = [
    "EmployeeID", "EmployeeName"
]

In [0]:
raw_employees_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("encoding", "UTF-8") \
    .option("multiLine", "true") \
    .option("mode", "DROPMALFORMED") \
    .load("file:/dbfs/tmp/github_csv/employees.csv")

In [0]:
employees_df = raw_employees_df.toDF(*clean_columns_employees)

# Clean up potential encoding artifacts in employee columns
employees_df = employees_df.select(
    regexp_replace(trim(col("EmployeeID").cast("string")), r'[^0-9]', "").cast("int").alias("EmployeeID"),
    regexp_replace(trim(col("EmployeeName")), '"', "").alias("EmployeeName")
)

employees_df.printSchema()
employees_df.show(5)

root
 |-- EmployeeID: integer (nullable = true)
 |-- EmployeeName: string (nullable = true)

+----------+------------+
|EmployeeID|EmployeeName|
+----------+------------+
|         1|       E 1  |
|         2|       E 2  |
|         3|       E 3  |
|         5|       E 4  |
|         6|       E 5  |
+----------+------------+
only showing top 5 rows



## 🧹 2. – Initial Data Quality Checks

Before diving into modeling, we begin with an early evaluation of the raw data to ensure its reliability and consistency. These checks help us make informed decisions when shaping our pipeline.

---

###Subsections:

- **2.1 – 🔍 Quick Data Quality Audit:** Assesses nulls, duplicates, and column uniqueness across all datasets.
- **2.2 – 🧼 Null Value Analysis & Data Cleaning:** Reviews missing values in detail and documents decisions on drops, transformations, or retention based on business logic.

### 2.1 - 🔍 Quick Data Quality Audit (Nulls, Duplicates & Uniqueness)

We define a helper function to automatically review key quality indicators across each dataset:
- Total number of rows
- Number of duplicate rows
- Null value count per column
- Number of distinct values per column

This gives a clear diagnostic of the integrity of each file before we begin our data cleaning strategy.

In [0]:
def quick_check_all(df, df_name="Unnamed DataFrame"):
    print(f"\n📊 Quality Check for: {df_name}")
    print("-" * 50)
    
    total_rows = df.count()
    print(f"Total Rows: {total_rows}")
    
    # Check for duplicate full rows
    duplicate_rows = total_rows - df.dropDuplicates().count()
    print(f"Duplicate Rows: {duplicate_rows}\n")
    
    for col_name in df.columns:
        nulls = df.filter(col(col_name).isNull()).count()
        distinct_vals = df.select(col_name).distinct().count()
        
        print(f"🔹 Column: {col_name}")
        print(f"   - Nulls: {nulls}")
        print(f"   - Distinct Values: {distinct_vals}")
        print("-" * 30)

for i in range(len(all_dfs)):
    quick_check_all(all_dfs[i], all_dfs_names[i])



📊 Quality Check for: customers_df
--------------------------------------------------
Total Rows: 225
Duplicate Rows: 0

🔹 Column: CustomerID
   - Nulls: 0
   - Distinct Values: 225
------------------------------
🔹 Column: CustomerName
   - Nulls: 0
   - Distinct Values: 225
------------------------------
🔹 Column: Region
   - Nulls: 0
   - Distinct Values: 87
------------------------------
🔹 Column: Country
   - Nulls: 0
   - Distinct Values: 2
------------------------------
🔹 Column: PriceCategory
   - Nulls: 0
   - Distinct Values: 7
------------------------------
🔹 Column: CustomerClass
   - Nulls: 0
   - Distinct Values: 7
------------------------------
🔹 Column: LeadSource
   - Nulls: 4
   - Distinct Values: 9
------------------------------
🔹 Column: Discontinued
   - Nulls: 0
   - Distinct Values: 2
------------------------------

📊 Quality Check for: employees_df
--------------------------------------------------
Total Rows: 15
Duplicate Rows: 0

🔹 Column: ��" E m p l o y e e I

### 2.2 - 🧼 Null Value Analysis & Data Cleaning Decisions

This section identifies columns across all datasets that contain null values. We present the total number of nulls, their percentage relative to the dataset, and the number of distinct (non-null) values for additional context.

Each case is briefly analyzed to determine whether nulls are tolerable (e.g., due to business logic) or problematic (e.g., in foreign key columns), helping guide our cleaning decisions and what to drop in the next step.

---

#### 📦 `orders_df`
- **ShippingMethodID**  
  - 🔸 Nulls: 8 out of 2,286 rows  
  - 🔸 Distinct values (non-null): 3  
  - 🧠 This is a very small portion of the dataset (~0.35%). These nulls likely indicate incomplete records. Since `ShippingMethodID` is a foreign key used for joins, we'll drop these 8 rows — doing so won't meaningfully impact the overall data quality or analysis.


---

#### 📦 `purchase_orders_df`
- **ShippingMethodID**  
  - 🔸 Nulls: 5 out of 232 rows  
  - 🔸 Distinct values (non-null): 4  
  - 🧠 These nulls represent just over 2% of the dataset. We'll also drop these rows to ensure clean joins with shipping method information, maintaining consistency across the pipeline.

---

#### 📦 `payments_df`
- **PaymentMethodID**  
  - 🔸 Nulls: 1 out of 686 rows  
  - 🔸 Distinct values: 3  
- **PaymentAmount**  
  - 🔸 Nulls: 1 out of 686 rows  
  - 🔸 Distinct values: 673  
  - 🧠 Both of these fields are essential: the payment method identifies how the transaction was made, and the payment amount represents its value. Rows with nulls in either field are incomplete and unusable, and will be dropped.

---

#### 📦 `products_df`
- **Color**  
  - 🔸 Nulls: 4,107 out of 4,183 rows (!!)  
  - 🔸 Distinct values: 19  
  - 🧠 This column is clearly not maintained — over 98% of values are missing. It offers no reliable insight and the column will be dropped entirely. For future color analysis, this field would need to be properly populated.
- **Size**  
  - 🔸 Nulls: 14 out of 4,183 rows  
  - 🔸 Distinct values: 25  
  - 🧠 While not as extreme as `Color`, this is still a small number of missing values. Since we don’t have a strong reason to preserve these specific rows, we'll drop them to maintain integrity for size-based queries.
- **ModelDescription**  
  - 🔸 Nulls: 10 out of 4,183 rows  
  - 🔸 Distinct values (non-null): 106  
- **FabricDescription**  
  - 🔸 Nulls: 13 out of 4,183 rows  
  - 🔸 Distinct values (non-null): 335  
  - 🧠 These columns appear to be optional descriptive fields, and the number of nulls is very small. We'll keep them for now as they could enrich analysis, especially for product descriptions or marketing insights.

---

#### 📦 `inventory_transactions_df`
- **TransactionDate**  
  - 🔸 Nulls: 1 out of 20,951 rows  
  - 🔸 Distinct values: 181  
  - 🧠 The single null should be removed.
- **UnitPurchasePrice**  
  - 🔸 Nulls: 1,151  
  - 🔸 Distinct values: 253  
  - 🧠 This value only applies to purchase transactions — it's the cost per unit bought. Other transaction types (like transfers or corrections) don’t involve purchases, so the field is left blank on purpose. These nulls are expected and will be retained.
- **QuantityOrdered**  
  - 🔸 Nulls: 2,195 out of 20,951 rows  
  - 🔸 Distinct values (non-null): 269  
- **QuantityReceived**  
  - 🔸 Nulls: 2,227 out of 20,951 rows  
  - 🔸 Distinct values (non-null): 269  
  - 🧠 Like `UnitPurchasePrice`, these fields are only meaningful in a subset of transactions. Their null presence reflects business logic, not missing data, so we will retain them.
- **MissingID**  
  - 🔸 Nulls: 18,757 out of 20,951 rows  
  - 🔸 Distinct values (non-null): 28  
- **QuantityMissing**  
  - 🔸 Nulls: 18,757 out of 20,951 rows  
  - 🔸 Distinct values (non-null): 94  
  - 🧠 These two fields are specific to loss/missing item transactions and are only populated when relevant. Their high null count is expected and does not indicate a data quality issue. We'll keep them.

---

> 🧹 **Next Step:** Based on the findings above, the next code cell applies our cleaning strategy by dropping specific rows or columns where null values are not acceptable.


In [0]:
#Cleaning the dataframes as mentioned above:

# ---------------------------------------------
# 📦 Cleaning: orders_df
# Drop rows with null ShippingMethodID
# ---------------------------------------------
initial = orders_df.count()
orders_df = orders_df.na.drop(subset=["ShippingMethodID"])
final = orders_df.count()
print(f"orders_df → Dropped {initial - final} rows ({(initial - final)/initial:.2%})")

# ---------------------------------------------
# 📦 Cleaning: purchase_orders_df
# Drop rows with null ShippingMethodID
# ---------------------------------------------
initial = purchase_orders_df.count()
purchase_orders_df = purchase_orders_df.na.drop(subset=["ShippingMethodID"])
final = purchase_orders_df.count()
print(f"purchase_orders_df → Dropped {initial - final} rows ({(initial - final)/initial:.2%})")

# ---------------------------------------------
# 📦 Cleaning: payments_df
# Drop rows with null PaymentMethodID or PaymentAmount
# ---------------------------------------------
initial = payments_df.count()
payments_df = payments_df.na.drop(subset=["PaymentMethodID", "PaymentAmount"])
final = payments_df.count()
print(f"payments_df → Dropped {initial - final} rows ({(initial - final)/initial:.2%})")

# ---------------------------------------------
# 📦 Cleaning: products_df
# Drop 'Color' column (too many nulls)
# Drop rows with null Size or FabricDescription
# ---------------------------------------------
products_df = products_df.drop("Color")  # Drop the column
initial = products_df.count()
products_df = products_df.na.drop(subset=["Size", "FabricDescription"])
final = products_df.count()
print(f"products_df → Dropped {initial - final} rows ({(initial - final)/initial:.2%})")

# ---------------------------------------------
# 📦 Cleaning: inventory_transactions_df
# Drop row with null TransactionDate (only 1 row)
# ---------------------------------------------
initial = inventory_transactions_df.count()
inventory_transactions_df = inventory_transactions_df.na.drop(subset=["TransactionDate"])
final = inventory_transactions_df.count()
print(f"inventory_transactions_df → Dropped {initial - final} rows ({(initial - final)/initial:.2%})")

orders_df → Dropped 8 rows (0.35%)
purchase_orders_df → Dropped 5 rows (2.16%)
payments_df → Dropped 2 rows (0.29%)
products_df → Dropped 14 rows (0.33%)
inventory_transactions_df → Dropped 1 rows (0.00%)


## 🧱 3. – Building the Enriched Fact Table

With the cleaned datasets prepared in the previous section, we now begin constructing the core **transactional table** that will later become our fact table in the star schema.

This process involves:

- **Merging the `orders` and `order_details` tables** to bring together general order info with item-level purchase data.
- **Enriching the combined dataset** (`fact_sales_df`) with relevant contextual attributes by joining it with other DataFrames: customers, products, employees, shipping methods, and payment-related information.
- Ensuring we **preserve all valid sales transactions**, even when some dimensions have missing or incomplete data, by using appropriate `left joins`.

By the end of this step, we will have a single, fully enriched DataFrame (`fact_sales_df`) containing all the relevant fields needed to support both analytical queries and the construction of a clean, modular star schema.

Each enrichment join is carefully structured and documented for transparency and scalability.

---

### Subsections:

- **3.1 – 📦 Create Base Fact Table (Orders + Order Details)**
- **3.2 – 👥 Enrich Fact Table with Customer and Employee Information**
- **3.3 – 🚚 Enrich Fact Table with Product and Shipping Information**
- **3.4 – 💳 Enrich Fact Table with Payment Information**

### 3.1 – 📦 Create Base Fact Table (Orders + Order Details)

In this step, we begin constructing the central **fact table** for the sales star schema. We perform an inner join between `orders_df` and `order_details_df` using the shared key `OrderID`.

This merge allows us to combine general order-level data (like customer, employee, shipping, and dates) with product-specific metrics (like quantity sold and unit sales price). The resulting table forms the foundation for revenue analysis and future joins with dimension tables.

In [0]:
# -----------------------------------------------
# 📦 Join orders_df and order_details_df
# Purpose: Combine general order info with product-level sales details
# -----------------------------------------------

fact_sales_df = orders_df.join(
    order_details_df,
    on="OrderID",  # shared key
    how="inner"    # inner join keeps only matching records
)

# Check the result
fact_sales_df.printSchema()
fact_sales_df.show(5)
print(f"✅ fact_sales_df created with {fact_sales_df.count()} rows")


root
 |-- OrderID: integer (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- EmployeeID: integer (nullable = true)
 |-- ShippingMethodID: integer (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- ShipDate: date (nullable = true)
 |-- FreightCharge: double (nullable = true)
 |-- OrderDetailID: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- QuantitySold: integer (nullable = true)
 |-- UnitSalesPrice: double (nullable = true)

+-------+----------+----------+----------------+----------+----------+-------------+-------------+---------+------------+--------------+
|OrderID|CustomerID|EmployeeID|ShippingMethodID| OrderDate|  ShipDate|FreightCharge|OrderDetailID|ProductID|QuantitySold|UnitSalesPrice|
+-------+----------+----------+----------------+----------+----------+-------------+-------------+---------+------------+--------------+
|      2|         1|         1|               1|2003-07-10|2003-07-10|          0.0|            2|      955

### 3.2 – 👥 Enrich Fact Table with Customer and Employee Information

We continue building the `fact_sales_df` by performing `left joins` to add customer and employee metadata from their respective dimension tables.

- Joining with `customers_df` via `CustomerID` allows us to bring in attributes like region, lead source, and customer class.
- Joining with `employees_df` via `EmployeeID` associates each order with the salesperson responsible.

These additions enable deeper segmentation and performance analysis across clients and internal teams while preserving all order-level records in the fact table.

In [0]:
# ----------------------------------------------------
# 📦 Join fact_sales_df with customers_df (on CustomerID)
# ----------------------------------------------------
fact_sales_df = fact_sales_df.join(
    customers_df,           # Dimension table
    on="CustomerID",        # Foreign key in fact table
    how="left"              # Preserve all sales, even if no customer info
)

# ----------------------------------------------------
# 📦 Join fact_sales_df with employees_df (on EmployeeID)
# ----------------------------------------------------
fact_sales_df = fact_sales_df.join(
    employees_df,           # Dimension table
    on="EmployeeID",        # Foreign key in fact table
    how="left"              # Preserve all sales, even if no employee info
)

# ✅ Preview the updated fact table
fact_sales_df.printSchema()
fact_sales_df.show(5, truncate=False)
print(f"✅ fact_sales_df updated with customer and product details. Total rows: {fact_sales_df.count()}")


root
 |-- EmployeeID: integer (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- OrderID: integer (nullable = true)
 |-- ShippingMethodID: integer (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- ShipDate: date (nullable = true)
 |-- FreightCharge: double (nullable = true)
 |-- OrderDetailID: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- QuantitySold: integer (nullable = true)
 |-- UnitSalesPrice: double (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- PriceCategory: integer (nullable = true)
 |-- CustomerClass: string (nullable = true)
 |-- LeadSource: string (nullable = true)
 |-- Discontinued: integer (nullable = true)
 |-- EmployeeName: string (nullable = true)

+----------+----------+-------+----------------+----------+----------+-------------+-------------+---------+------------+--------------+------------+------+------------------+

### 3.3 – 🚚 Enrich Fact Table with Product and Shipping Information

In this step, we continue enriching the `fact_sales_df` by joining additional contextual data:

- The `products_df` is joined via `ProductID`, adding valuable attributes such as category, gender, model description, and size. This enhances product-level sales analysis and segmentation.
- The `shipping_methods_df` is joined using `ShippingMethodID`, allowing us to evaluate logistics and shipping preferences at scale.

These enrichments help us analyze performance not just by customer or employee, but also by product attributes and delivery methods.

In [0]:
# ----------------------------------------------------
# 📦 Join fact_sales_df with products_df (on ProductID)
# ----------------------------------------------------
fact_sales_df = fact_sales_df.join(
    products_df,            # Dimension table
    on="ProductID",         # Foreign key in fact table
    how="left"              # Preserve all sales, even if no product info
)

# ---------------------------------------------------------------
# 📦 Join fact_sales_df with shipping_methods_df (on ShippingMethodID)
# ---------------------------------------------------------------
fact_sales_df = fact_sales_df.join(
    shipping_methods_df,    # Dimension table
    on="ShippingMethodID",  # Foreign key in fact table
    how="left"              # Preserve all sales, even if no shipping info
)

# ✅ Preview the final enriched fact table
fact_sales_df.printSchema()
fact_sales_df.show(5, truncate=False)
print(f"✅ fact_sales_df fully enriched with employees and shipping info. Total rows: {fact_sales_df.count()}")


root
 |-- ShippingMethodID: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- EmployeeID: integer (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- OrderID: integer (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- ShipDate: date (nullable = true)
 |-- FreightCharge: double (nullable = true)
 |-- OrderDetailID: integer (nullable = true)
 |-- QuantitySold: integer (nullable = true)
 |-- UnitSalesPrice: double (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- PriceCategory: integer (nullable = true)
 |-- CustomerClass: string (nullable = true)
 |-- LeadSource: string (nullable = true)
 |-- Discontinued: integer (nullable = true)
 |-- EmployeeName: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- ModelDescription: string (nullable = true)
 |-- FabricDescription: string (nullable = true)
 |-- Category: string (nullable = tr

### 3.4 – 💳 Enrich Fact Table with Payment Information

In this step, we finalize the enrichment of the `fact_sales_df` by incorporating payment-related data:

- First, we join `payments_df` via `OrderID` to bring in payment-specific records. This allows us to understand which sales have associated payment information.
- Then, we join `payment_methods_df` using `PaymentMethodID` to map each payment to its respective method (e.g. card, transfer, etc.).

These joins support financial analysis and can help assess how different payment methods impact revenue or order behavior.

In [0]:
# ------------------------------------------------------------
# 📦 Join fact_sales_df with payments_df (on OrderID)
# (to retrieve PaymentMethodID from payments)
# ------------------------------------------------------------
fact_sales_df = fact_sales_df.join(
    payments_df,            # Dimension table 
    on="OrderID", 
    how="left"              # Preserve all sales, even if no payment info
)

# ----------------------------------------------------------------------
# 📦 Join fact_sales_df with payment_methods_df (on PaymentMethodID)
# ----------------------------------------------------------------------
fact_sales_df = fact_sales_df.join(
    payment_methods_df,      # Dimension table
    on="PaymentMethodID",    # Foreign key now present from previous join
    how="left"               # Preserve all sales, even if no payment method info
)

# ✅ Preview the final enriched fact table
fact_sales_df.printSchema()
fact_sales_df.show(5, truncate=False)
print(f"✅ fact_sales_df fully enriched with employees and shipping info. Total rows: {fact_sales_df.count()}")

root
 |-- PaymentMethodID: integer (nullable = true)
 |-- OrderID: integer (nullable = true)
 |-- ShippingMethodID: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- EmployeeID: integer (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- ShipDate: date (nullable = true)
 |-- FreightCharge: double (nullable = true)
 |-- OrderDetailID: integer (nullable = true)
 |-- QuantitySold: integer (nullable = true)
 |-- UnitSalesPrice: double (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- PriceCategory: integer (nullable = true)
 |-- CustomerClass: string (nullable = true)
 |-- LeadSource: string (nullable = true)
 |-- Discontinued: integer (nullable = true)
 |-- EmployeeName: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- ModelDescription: string (nullable = true)
 |-- FabricDescription: string (nulla

## 4. – ⭐ Building the Star Schema: Dimensions & Fact Table

With the enriched `fact_sales_df` fully prepared, we now transition from our exploratory format to a formal **star schema**. This modeling approach is essential for efficient querying, data warehousing, and BI tool integration.

In this step, we will:
- Extract and clean **dimension tables** from `fact_sales_df`
- Create a compact, analytical **fact table** with only numerical measures and foreign keys
- Ensure schema consistency across all joins

---

### 🧱 Star Schema Overview

The structure consists of one central **fact table** and multiple supporting **dimension tables**, each focused on a key aspect of the business.

Each dimension captures a specific **descriptive entity** involved in the sales process — such as customers, products, employees, or shipping methods. These tables reduce redundancy and bring context for filtering, aggregation, and slicing in downstream queries.

| Table Name             | Type       | Description                                                              |
|------------------------|------------|--------------------------------------------------------------------------|
| `fact_sales`           | Fact        | Central table storing transaction-level data: foreign keys + numerical measures |
| `dim_customers`        | Dimension   | Describes each customer, including region, class, price category, and more |
| `dim_products`         | Dimension   | Details each product, including category, size, material, and status     |
| `dim_employees`        | Dimension   | Identifies the employee linked to the sale                               |
| `dim_shipping_methods` | Dimension   | Describes how the order was delivered                                    |
| `dim_payments`         | Dimension   | Records payment transaction metadata including method and amount         |
| `dim_date`             | Dimension   | A calendar dimension used for time-based analysis                        |

---

### 🛠 Methodology for Dimension Creation

Each dimension follows a standardized creation process:
- Select the relevant attributes from `fact_sales_df` or staging DataFrames
- Drop duplicate records using the primary identifier
- Sort for clarity and consistency
- Track rows removed for transparency
- Preview a sample of the results

---

### ✅ Final Step: Fact Table Creation

Once all dimension tables are complete, we reduce `fact_sales_df` to a **clean analytical fact table**, preserving only:
- **Foreign keys** pointing to the dimensions
- **Quantitative measures** like quantity sold, sales price, and freight charges
- **Date keys** that allow time-series aggregation

This results in a fully structured **star schema** ready for SQL queries and business insight extraction.

---

### Subsections:

- **4.1 – 🧑‍💼 Create `dim_customers`: Customer Dimension Table**  
- **4.2 – 🛍️ Create `dim_products`: Product Dimension Table**  
- **4.3 – 👤 Create `dim_employees`: Employees Dimension Table**  
- **4.4 – 🚚 Create `dim_shipping_methods`: Shipping Methods Dimension Table**  
- **4.5 – 💳 Create `dim_payments`: Payments Dimension Table**  
- **4.6 – 📅 Create `dim_date`: Date Dimension Table**  
- **4.7 – 🧾 Create Final `fact_sales`: Fact Table**

> Once complete, the star schema will serve as the foundation for all business questions and advanced SQL analysis in the next phase.

### 4.1 – 🧑‍💼 Create `dim_customers`: Customer Dimension Table

In this step, we extract unique customer-related information from the `fact_sales_df` to create the `dim_customers` table. This dimension captures key attributes about each customer that can be used to analyze behavior, segment the client base, and inform strategic decisions.

The dimension is created by selecting only relevant columns, removing duplicates based on `CustomerID`, and sorting for readability.

| Attribute       | Type     | Description                                               |
|----------------|----------|-----------------------------------------------------------|
| CustomerID     | Integer  | Unique identifier for each customer                       |
| CustomerName   | String   | Name or label assigned to the customer                    |
| Region         | String   | Geographical region of the customer                       |
| Country        | String   | Country where the customer operates                       |
| PriceCategory  | Integer  | Pricing tier applied to the customer                      |
| CustomerClass  | String   | Type or profile of customer                               |
| LeadSource     | String   | Origin of the customer relationship                       |
| Discontinued   | Integer  | 0 = active customer, 1 = discontinued or inactive         |

This structure will support future customer-level insights.

In [0]:
# -----------------------------------------------
# 📦 Create dim_customers from fact_sales_df
# -----------------------------------------------
dim_customers = fact_sales_df.select(
    "CustomerID",          # Unique customer identifier
    "CustomerName",        # Name of customer
    "Region",              # Geographical region
    "Country",             # Country of origin
    "PriceCategory",       # Pricing tier
    "CustomerClass",       # Type/category of customer
    "LeadSource",          # How customer was acquired
    "Discontinued"         # If the customer account is still active
).dropDuplicates(["CustomerID"])

# Sort for readability
dim_customers = dim_customers.orderBy("CustomerID")

# Track dropped rows
dropped = fact_sales_df.select("CustomerID").distinct().count() - dim_customers.count()
print(f"🧹 dim_customers → {dropped} duplicate rows dropped based on CustomerID")


# ✅ Preview the result
dim_customers.printSchema()
dim_customers.show(5, truncate=False)
print(f"✅ dim_customers created with {dim_customers.count()} unique customers.")

🧹 dim_customers → 0 duplicate rows dropped based on CustomerID
root
 |-- CustomerID: integer (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- PriceCategory: integer (nullable = true)
 |-- CustomerClass: string (nullable = true)
 |-- LeadSource: string (nullable = true)
 |-- Discontinued: integer (nullable = true)

+----------+------------+------+------------------+-------------+------------------------+------------------------------+------------+
|CustomerID|CustomerName|Region|Country           |PriceCategory|CustomerClass           |LeadSource                    |Discontinued|
+----------+------------+------+------------------+-------------+------------------------+------------------------------+------------+
|1         |C1          |Moscow|Russian Federation|1            |Large-Scale Wholesaler-1|Referral by the Central Office|0           |
|2         |C2          |Moscow|Russian Federatio

### 4.2 – 🛍️ Create `dim_products`: Product Dimension Table

This step extracts unique product details from the `fact_sales_df` to construct the `dim_products` dimension. Each entry represents a distinct product and includes attributes that support analysis across product type, target audience, pricing, and lifecycle status.

| Attribute         | Type     | Description                                                     |
|------------------|----------|-----------------------------------------------------------------|
| ProductID        | Integer  | Unique identifier for the product                               |
| ProductName      | String   | Internal code or name for the product                           |
| ModelDescription | String   | Short model-level label or design code                          |
| FabricDescription| String   | Description of the material/fabric                              |
| Category         | String   | Category like briefs, undershirts, socks, etc.                  |
| Gender           | String   | Target audience by gender and age                               |
| ProductLine      | String   | Sub-brand or product family                                     |
| Weight           | Integer  | Weight of a unit in grams                                       |
| Size             | String   | Size label                                                      |
| PackSize         | String   | Units per sales pack                                            |
| Status           | String   | Lifecycle status                                                |
| InventoryDate    | Date     | Date the item was added to inventory                            |
| PurchasePrice    | Double   | Internal cost to purchase the item                              |

The product dimension enables product-level filtering and comparison across fabric types, sizes, gender targeting, and lifecycle stages.

In [0]:
# -----------------------------------------------
# 📦 Create dim_products from fact_sales_df
# -----------------------------------------------
dim_products = fact_sales_df.select(
    "ProductID",          # Unique product identifier
    "ProductName",        # Product name
    "ModelDescription",   # Model-level descriptor
    "FabricDescription",  # Fabric type or quality
    "Category",           # Underwear, sock, etc.
    "Gender",             # Target gender
    "ProductLine",        # Sub-brand or series
    "Weight",             # Weight of product
    "Size",               # Size variant
    "PackSize",           # Units per pack
    "Status",             # Active/Inactive
    "InventoryDate",      # When it was stocked
    "PurchasePrice"       # Internal purchase cost
).dropDuplicates(["ProductID"])

# Optional: sort for visual clarity
dim_products = dim_products.orderBy("ProductID")

# Track duplicate drops
dropped = fact_sales_df.select("ProductID").distinct().count() - dim_products.count()
print(f"🧹 dim_products → {dropped} duplicate rows dropped based on ProductID")

# ✅ Preview
dim_products.printSchema()
dim_products.show(5, truncate=False)
print(f"✅ dim_products created with {dim_products.count()} unique products.")

🧹 dim_products → 0 duplicate rows dropped based on ProductID
root
 |-- ProductID: integer (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- ModelDescription: string (nullable = true)
 |-- FabricDescription: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- ProductLine: string (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- PackSize: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- InventoryDate: date (nullable = true)
 |-- PurchasePrice: double (nullable = true)

+---------+-----------+----------------+-----------------+-----------+------------------+-----------+------+----+--------+-----------------+-------------+-------------+
|ProductID|ProductName|ModelDescription|FabricDescription|Category   |Gender            |ProductLine|Weight|Size|PackSize|Status           |InventoryDate|PurchasePrice|
+---------+-----------+----------------+-----------

### 4.3 – 👤 Create Dimension Table: Employees

The `dim_employees` table stores information about the employees involved in sales operations. It is created by selecting distinct employee records from the cleaned `employees_df`.

This dimension allows us to attribute sales performance to individual employees and support future analyses on workforce impact and operational structure.

| Attribute      | Type   | Description                          |
|----------------|--------|--------------------------------------|
| EmployeeID     | integer| Unique identifier for employee       |
| EmployeeName   | string | Internal code/name                   |

In [0]:
# -----------------------------------------------
# 👤 Create dim_employees from employees_df
# -----------------------------------------------
dim_employees = employees_df.select(
    "EmployeeID",     # Unique employee identifier
    "EmployeeName"    # Employee code (e.g., E1, E2, ...)
).dropDuplicates(["EmployeeID"])

# Optional: sort for clarity
dim_employees = dim_employees.orderBy("EmployeeID")

# Track dropped rows (if any)
dropped = employees_df.select("EmployeeID").distinct().count() - dim_employees.count()
print(f"🧹 dim_employees → {dropped} duplicate rows dropped based on EmployeeID")

# ✅ Preview
dim_employees.printSchema()
dim_employees.show(5, truncate=False)
print(f"✅ dim_employees created with {dim_employees.count()} unique employees.")

🧹 dim_employees → 0 duplicate rows dropped based on EmployeeID
root
 |-- EmployeeID: integer (nullable = true)
 |-- EmployeeName: string (nullable = true)

+----------+------------+
|EmployeeID|EmployeeName|
+----------+------------+
|1         |  E 1       |
|2         |  E 2       |
|3         |  E 3       |
|5         |  E 4       |
|6         |  E 5       |
+----------+------------+
only showing top 5 rows

✅ dim_employees created with 16 unique employees.


### 4.4 – 🚚 Create Dimension Table: Shipping Methods

The `dim_shipping_methods` table catalogs all shipping methods available in the dataset. It is derived from the `fact_sales_df` by selecting unique combinations of `ShippingMethodID` and `ShippingMethod`.

This dimension supports analysis on logistics performance, preferences, and cost comparisons across different delivery methods.

| Attribute         | Type    | Description                              |
|-------------------|---------|------------------------------------------|
| ShippingMethodID  | int     | Unique identifier for the shipping method|
| ShippingMethod    | string  | Name or description of the method used   |

In [0]:
# -----------------------------------------------------------
# 🚚 Create dim_shipping_methods from fact_sales_df
# -----------------------------------------------------------
dim_shipping_methods = fact_sales_df.select(
    "ShippingMethodID",   # Unique identifier
    "ShippingMethod"      # Correct name of the column
).dropDuplicates(["ShippingMethodID"])

# Optional: sort for clarity
dim_shipping_methods = dim_shipping_methods.orderBy("ShippingMethodID")

# Track dropped rows (if any)
dropped = fact_sales_df.select("ShippingMethodID").distinct().count() - dim_shipping_methods.count()
print(f"🧹 dim_shipping_methods → {dropped} duplicate rows dropped based on ShippingMethodID")

# ✅ Preview
dim_shipping_methods.printSchema()
dim_shipping_methods.show(5, truncate=False)
print(f"✅ dim_shipping_methods created with {dim_shipping_methods.count()} unique entries.")

🧹 dim_shipping_methods → 0 duplicate rows dropped based on ShippingMethodID
root
 |-- ShippingMethodID: integer (nullable = true)
 |-- ShippingMethod: string (nullable = true)

+----------------+--------------+
|ShippingMethodID|ShippingMethod|
+----------------+--------------+
|1               |Ex Works      |
|3               |Container     |
+----------------+--------------+

✅ dim_shipping_methods created with 2 unique entries.


### 4.5 – 💳 Create Dimension Table: Payments

The `dim_payments` table stores detailed records of payment transactions associated with customer orders. Extracted from the original `payments_df`, it ensures each entry is uniquely identified and includes relevant metadata for payment analysis.

| Attribute        | Type     | Description                                      |
|------------------|----------|--------------------------------------------------|
| PaymentID        | int      | Unique identifier for the payment                |
| OrderID          | int      | Foreign key linking to the sales fact table      |
| PaymentMethodID  | int      | Payment Method identifyer                        |
| PaymentDate      | date     | The date the payment was recorded                |
| PaymentAmount    | float    | Total amount paid for the transaction (USD)      |

In [0]:
# -----------------------------------------------------------
# 💳 Create dim_payments from payments_df
# -----------------------------------------------------------
dim_payments = payments_df.select(
    "PaymentID",          # Unique identifier for the payment
    "OrderID",            # FK to fact_sales (or used for joins)
    "PaymentMethodID",    # FK to dim_payment_methods
    "PaymentDate",        # When the payment was made
    "PaymentAmount"       # How much was paid
).dropDuplicates(["PaymentID"])

# Optional: sort for clarity
dim_payments = dim_payments.orderBy("PaymentID")

# Track dropped rows (if any)
dropped = payments_df.select("PaymentID").distinct().count() - dim_payments.count()
print(f"🧹 dim_payments → {dropped} duplicate rows dropped based on PaymentID")

# ✅ Preview
dim_payments.printSchema()
dim_payments.show(5, truncate=False)
print(f"✅ dim_payments created with {dim_payments.count()} unique entries.")

🧹 dim_payments → 0 duplicate rows dropped based on PaymentID
root
 |-- PaymentID: integer (nullable = true)
 |-- OrderID: integer (nullable = true)
 |-- PaymentMethodID: integer (nullable = true)
 |-- PaymentDate: date (nullable = true)
 |-- PaymentAmount: string (nullable = true)

+---------+-------+---------------+-----------+-------------+
|PaymentID|OrderID|PaymentMethodID|PaymentDate|PaymentAmount|
+---------+-------+---------------+-----------+-------------+
|1        |2      |1              |2003-07-10 |603.50       |
|2        |4      |1              |2003-07-12 |1,288.00     |
|4        |5      |1              |2003-07-15 |1,800.00     |
|5        |6      |1              |2003-07-14 |389.50       |
|6        |7      |1              |2003-07-15 |1,479.00     |
+---------+-------+---------------+-----------+-------------+
only showing top 5 rows

✅ dim_payments created with 684 unique entries.


### 4.6 – 📅 Create Dimension Table: Date

The `dim_date` table serves as a calendar dimension that allows for rich time-based analysis, such as tracking sales by **month**, **weekday**, or **quarter**. It enables consistent filtering and grouping across fact tables by replacing raw date fields with descriptive foreign key relationships.

In this project, we extract all unique `OrderDate` and `ShipDate` values from the fact table, combine them into a single date key column, and enrich each date with temporal attributes. This setup supports advanced trend analysis such as **seasonality**, **weekday patterns**, or **shipping delays**.

Both `OrderDate` and `ShipDate` remain in the `fact_sales` table and act as **foreign keys** referencing `dim_date.DateKey`.

| Attribute     | Type     | Description                                      |
|---------------|----------|--------------------------------------------------|
| DateKey       | date     | Unique calendar date referenced in fact table    |
| Year          | integer      | Calendar year of the date                        |
| Month         | integer      | Numeric month (1–12)                             |
| Day           | integer      | Day of the month (1–31)                          |
| Quarter       | integer      | Calendar quarter (1–4)                           |
| WeekOfYear    | integer      | ISO week number of the year                      |
| DayOfWeek     | integer      | Numeric day of week (1=Sunday, 7=Saturday)       |
| WeekdayName   | string   | Name of the day (e.g., Monday, Tuesday)          |

> 📌 This dimension enhances flexibility in temporal reporting and aligns the sales pipeline around a unified time reference structure.

In [0]:
# -----------------------------------------------
# 📅 Extract unique dates from OrderDate and ShipDate
# -----------------------------------------------
order_dates = fact_sales_df.select(col("OrderDate").alias("DateKey")).distinct()
ship_dates = fact_sales_df.select(col("ShipDate").alias("DateKey")).distinct()

# Combine and deduplicate
dim_date = order_dates.union(ship_dates).distinct()

# -----------------------------------------------
# 📅 Add enriched date attributes
# -----------------------------------------------
dim_date = dim_date.withColumn("Year", year("DateKey")) \
                   .withColumn("Month", month("DateKey")) \
                   .withColumn("Day", dayofmonth("DateKey")) \
                   .withColumn("Quarter", quarter("DateKey")) \
                   .withColumn("WeekOfYear", weekofyear("DateKey")) \
                   .withColumn("DayOfWeek", dayofweek("DateKey")) \
                   .withColumn("WeekdayName", date_format("DateKey", "EEEE")) \
                   .orderBy("DateKey")

# ✅ Preview
dim_date.printSchema()
dim_date.show(5, truncate=False)
print(f"✅ dim_date created with {dim_date.count()} unique dates.")


root
 |-- DateKey: date (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- WeekOfYear: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- WeekdayName: string (nullable = true)

+----------+----+-----+---+-------+----------+---------+-----------+
|DateKey   |Year|Month|Day|Quarter|WeekOfYear|DayOfWeek|WeekdayName|
+----------+----+-----+---+-------+----------+---------+-----------+
|2001-10-11|2001|10   |11 |4      |41        |5        |Thursday   |
|2001-10-20|2001|10   |20 |4      |42        |7        |Saturday   |
|2002-05-16|2002|5    |16 |2      |20        |5        |Thursday   |
|2002-10-02|2002|10   |2  |4      |40        |4        |Wednesday  |
|2003-06-19|2003|6    |19 |2      |25        |5        |Thursday   |
+----------+----+-----+---+-------+----------+---------+-----------+
only showing top 5 rows

✅ dim_date created with 757 uniq

### 4.7 – 🧾 Create Final `fact_sales` Table

At this stage, we finalize the star schema by trimming down the enriched `fact_sales_df` to retain only the fields required for analysis. These include:

- **Foreign keys** linking to each dimension table
- **Quantitative metrics** such as quantity sold, sales price, and freight charge
- **Date keys** (order and ship) for time-based aggregation

We also perform a final cleanup step: ensuring that all rows in `fact_sales` have valid `ProductID`s with non-null names and categories. This removes incomplete or orphaned sales entries that would distort product-level insights.

---

#### ✅ Final Schema: `fact_sales`

| Column             | Type     | Description                                |
|--------------------|----------|--------------------------------------------|
| `OrderID`          | integer  | Unique identifier for the order (PK)       |
| `CustomerID`       | integer  | FK to `dim_customers`                      |
| `ProductID`        | integer  | FK to `dim_products`                       |
| `EmployeeID`       | integer  | FK to `dim_employees`                      |
| `ShippingMethodID` | integer  | FK to `dim_shipping_methods`               |
| `PaymentMethodID`  | integer  | FK to `dim_payment_methods`                |
| `OrderDateKey`     | date     | FK to `dim_date` (order placed)            |
| `ShipDateKey`      | date     | FK to `dim_date` (order shipped)           |
| `QuantitySold`     | integer  | Number of items sold in the order          |
| `UnitSalesPrice`   | double   | Price per item sold                        |
| `FreightCharge`    | double   | Shipping charge applied to the order       |

---

This clean, optimized fact table is ready for querying using Spark SQL and supports multidimensional analysis across customers, products, employees, regions, and time.

In [0]:
# -----------------------------------------------------------
# ✂️ Select only necessary fields for the fact_sales table
# -----------------------------------------------------------
fact_sales = fact_sales_df.select(
    "OrderID",           # Unique order identifier
    "CustomerID",        # FK to dim_customers
    "ProductID",         # FK to dim_products
    "EmployeeID",        # FK to dim_employees
    "ShippingMethodID",  # FK to dim_shipping_methods
    "PaymentMethodID",   # FK to dim_payment_methods
    col("OrderDate").alias("OrderDateKey"),  # FK to dim_date
    col("ShipDate").alias("ShipDateKey"),    # FK to dim_date
    "QuantitySold",      # Sales quantity
    "UnitSalesPrice",    # Unit price sold
    "FreightCharge"      # Shipping cost
)

# ✂️ Drop rows from fact_sales that no longer have a matching product in dim_products
valid_product_ids = dim_products.filter(
    (col("ProductName").isNotNull()) & (col("Category").isNotNull())
).select("ProductID").distinct()

initial = fact_sales.count()
fact_sales = fact_sales.join(valid_product_ids, on="ProductID", how="inner")
final = fact_sales.count()

print(f"🧹 Cleaned fact_sales → Dropped {initial - final} rows ({(initial - final)/initial:.2%}) due to missing products in dim_products")

# ✅ Preview the cleaned fact table
fact_sales.printSchema()
fact_sales.show(5, truncate=False)
print(f"✅ Final fact_sales table created with {fact_sales.count()} rows.")

🧹 Cleaned fact_sales → Dropped 218 rows (0.20%) due to missing products in dim_products
root
 |-- ProductID: integer (nullable = true)
 |-- OrderID: integer (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- EmployeeID: integer (nullable = true)
 |-- ShippingMethodID: integer (nullable = true)
 |-- PaymentMethodID: integer (nullable = true)
 |-- OrderDateKey: date (nullable = true)
 |-- ShipDateKey: date (nullable = true)
 |-- QuantitySold: integer (nullable = true)
 |-- UnitSalesPrice: double (nullable = true)
 |-- FreightCharge: double (nullable = true)

+---------+-------+----------+----------+----------------+---------------+------------+-----------+------------+--------------+-------------+
|ProductID|OrderID|CustomerID|EmployeeID|ShippingMethodID|PaymentMethodID|OrderDateKey|ShipDateKey|QuantitySold|UnitSalesPrice|FreightCharge|
+---------+-------+----------+----------+----------------+---------------+------------+-----------+------------+--------------+-----------

### ⭐ Star Schema Completed

With the creation of the final `fact_sales` table and all supporting dimension tables, our star schema is now fully constructed and ready for analysis.

The structure includes:
- A clean central fact table containing only keys and measurable facts
- Fully normalized dimension tables providing semantic context for customers, products, employees, shipping methods, payment types, and dates

This schema is now optimized for running efficient queries, powering dashboards, and generating business insights.

##5. – 📊 Business Questions & Strategic Insights

With the star schema fully modeled and cleaned, we now enter the **analytical phase** of the project. This section is dedicated to uncovering patterns and opportunities through structured SQL queries built on top of our curated data model.

In this section, we will:
- Pose key business questions that this dataset can help answer
- Use SQL queries over our star schema to extract meaningful insights
- Support future decision-making based on patterns in sales, customers, time, and operations

Each question will be preceded by a short explanation and followed by the corresponding SQL query and result.

---

### Areas of Analysis:

- Sales and payments  
- Product performance  
- Regional trends  
- Customer segments  
- Key accounts  
- Employee contribution  
- Time-based trends  
- Shipping operations

---

### Methodology Notes:

- Our **primary revenue metric** is calculated directly from the sales data:  
  `Revenue = QuantitySold * UnitSalesPrice`

- The payments dataset is only partially complete (~28% of orders include payment records), so it is **not used as the main financial source**. Instead, it supports analysis where relevant and noted.

---

### Subsections:

- **5.1 – 🧾 Sales Orders vs. Paid Orders**
- **5.2 – 🧍 Revenue by Product Gender**
- **5.3 – 🩲 Revenue by Product Category**
- **5.4 – 🌍 Regional Revenue Breakdown — Top 10 Cities by Total Revenue**
- **5.5 – 🔍 Revenue by Lead Source**
- **5.6 – 🧩 Revenue by Customer Class**
- **5.7 – 🔁 Revenue by Customer Status (Active vs. Discontinued)**
- **5.8 – 🏆 Top 10 Customers – Performance Deep Dive**
- **5.9 – 🧑‍💼 Employee Performance Analysis**
- **5.10 – 📆 Revenue Trend Over Time**
- **5.11 – 💰 Average Order Value (AOV) by Customer Class**
- **5.12 – 🔁 Revenue by Customer Loyalty**
- **5.13 – 🚚 Shipping Method Overview**

> 💡 Each subsection explores a business challenge or KPI, presents the query logic, and delivers commentary on the output.

### 5.0 – 🧠 Registering Temp Views for SQL Queries

Before we begin exploring the business questions, we register each dimension and fact table as a **temporary view** using `createOrReplaceTempView()`. This allows us to write SQL queries directly on top of Spark DataFrames using the `spark.sql()` interface.

The following views are now available for analysis:

- `fact_sales`
- `dim_customers`
- `dim_products`
- `dim_employees`
- `dim_shipping_methods`
- `dim_payments`
- `dim_date`

> This step bridges the gap between PySpark and SQL, enabling clean, readable queries throughout the insights section.

In [0]:
fact_sales.createOrReplaceTempView("fact_sales")
dim_customers.createOrReplaceTempView("dim_customers")
dim_products.createOrReplaceTempView("dim_products")
dim_employees.createOrReplaceTempView("dim_employees")
dim_shipping_methods.createOrReplaceTempView("dim_shipping_methods")
dim_payments.createOrReplaceTempView("dim_payments")
dim_date.createOrReplaceTempView("dim_date")

###5.1 🧾 Sales Orders vs. Paid Orders

| Metric             | Count |
|--------------------|-------|
| Total Sales Orders | 2,261 |
| Paid Orders        | 648   |

While payment data is available in the dataset, **only 648 out of 2,261 sales orders have corresponding payment records** — representing less than 29% of all transactions. This gap suggests either **missing data** or that some payments were recorded outside the scope of this dataset.

> ✅ **Conclusion**: For analytical consistency and completeness, **revenue will be calculated using the sales records** (`QuantitySold × UnitSalesPrice`) rather than relying on payment data.

This approach ensures we can account for **100% of recorded sales**, providing a more accurate and holistic view of business performance.

In [0]:
%sql
-- 🧾 Count of Total Sales Orders vs. Paid Orders
SELECT
  COUNT(DISTINCT f.OrderID) AS TotalSalesOrders,
  COUNT(DISTINCT p.OrderID) AS PaidOrders
FROM fact_sales f
LEFT JOIN dim_payments p ON f.OrderID = p.OrderID;

TotalSalesOrders,PaidOrders
2261,648


### 5.2 - 🧍 Revenue by Product Gender

This analysis reveals how product sales are distributed across gendered product lines.

| Gender                | Total Revenue (USD) | % of Total Revenue |
|------------------------|--------------------|---------------------|
| Women's Panties        | 3,190,048.55       | 78.38%              |
| Men's Briefs           |   282,155.82       | 6.93%               |
| Girls' Panties         |   230,337.50       | 5.66%               |
| Men's Undershirts      |   146,654.90       | 3.60%               |
| Boys' Briefs           |    78,415.40       | 1.93%               |
| Women's Undershirts    |    62,103.50       | 1.53%               |
| Girls' Undershirts     |    42,553.00       | 1.05%               |
| Children's Socks       |    14,043.60       | 0.35%               |
| Boys' Undershirts      |    11,545.20       | 0.28%               |
| Men's Socks            |     7,162.70       | 0.18%               |
| Women's Socks          |     5,036.50       | 0.12%               |

- The business is **highly specialized in female underwear**, particularly **Women’s Panties**, which alone make up nearly 80% of total revenue.
- Other product categories contribute marginally, with **men’s and children’s lines showing limited market traction**.
- There is **room to explore men’s products**, but any expansion should be tested gradually and supported by data.
- **Sock and undershirt categories**, while not significant in revenue, still play a role in the broader **brand perception**.

> 📌 **Insight**: These complementary items — like socks and undershirts — **reinforce the brand's image of intimacy, comfort, and product completeness**. They help shape the narrative around quality and trust, supporting the main performers and offering customers a fuller lifestyle experience.

In [0]:
%sql
-- 🧍 Revenue by Product Gender
SELECT 
  p.Gender,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue,
  ROUND(
    100 * SUM(f.QuantitySold * f.UnitSalesPrice) / 
    SUM(SUM(f.QuantitySold * f.UnitSalesPrice)) OVER (),
    2
  ) AS RevenuePercentage
FROM fact_sales f
JOIN dim_products p ON f.ProductID = p.ProductID
GROUP BY p.Gender
ORDER BY TotalRevenue DESC;

Gender,TotalRevenue,RevenuePercentage
Women's Panties,3190048.55,78.38
Men's Briefs,282155.82,6.93
Girls' Panties,230337.5,5.66
Men's Undershirts,146654.9,3.6
Boys' Briefs,78415.4,1.93
Women's Undershirts,62103.5,1.53
Girls' Undershirts,42553.0,1.05
Children's Socks,14043.6,0.35
Boys' Undershirts,11545.2,0.28
Men's Socks,7162.7,0.18


###5.3 - 🩲 Revenue by Product Category

The analysis of total revenue by product category reveals a **clear specialization** in the company’s offering.

| Category                 | Total Revenue (R$) | % of Total Revenue |
|--------------------------|--------------------|---------------------|
| **Briefs**               | 3,365,131.27       | 85.58%              |
| Undershirts              |   262,931.50       | 6.69%               |
| Thongs                   |   184,980.40       | 4.71%               |
| Drawers                  |   160,898.10       | 4.09%               |
| Boxers                   |    67,536.50       | 1.72%               |
| Regular-Over the Calf    |    25,835.00       | 0.66%               |
| Slips                    |     2,336.10       | 0.06%               |
| Cuff-Over the Calf       |       407.80       | 0.01%               |

- **Briefs represent over 85% of total revenue**, clearly establishing them as the company’s core product.
- The remaining categories collectively contribute less than 15%, with some items (like socks and slips) barely registering in the overall picture.
- This product mix highlights the brand’s strategic focus on essentials and mass-volume items, with limited diversification into niche apparel.

> 📌 **Insight**: The company may consider doubling down on its most profitable segments while reassessing the ROI of maintaining minor product categories in inventory.

In [0]:
%sql
-- 🩲 Revenue by Product Category + % of Total
SELECT 
  p.Category,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue,
  ROUND(
    100 * SUM(f.QuantitySold * f.UnitSalesPrice) / 
    SUM(SUM(f.QuantitySold * f.UnitSalesPrice)) OVER (), 
    2
  ) AS RevenuePercentage
FROM fact_sales f
JOIN dim_products p ON f.ProductID = p.ProductID
GROUP BY p.Category
ORDER BY TotalRevenue DESC;

Category,TotalRevenue,RevenuePercentage
Briefs,3365131.27,82.68
Undershirts,262931.5,6.46
Thongs,184980.4,4.54
Drawers,160898.1,3.95
Boxers,67536.5,1.66
Regular-Over the Calf,25835.0,0.63
Slips,2336.1,0.06
Cuff-Over the Calf,407.8,0.01


Databricks visualization. Run in Databricks to view.

###5.4 - 🌍 Regional Revenue Breakdown — Top 10 Cities by Total Revenue

Understanding the geographic distribution of revenue helps the company identify where its most valuable customer bases are located and where additional market development might be worthwhile.

| Region         | Total Revenue (USD) | Revenue (%) | Orders | Avg. Order Value (USD) | Unique Customers | Avg. Revenue/Customer (USD) |
|----------------|----------------------|-------------|--------|-------------------------|-------------------|------------------------------|
| Moscow         | 1,878,887.57         | 46.16       | 1412   | 1,330.66                | 62                | 30,304.64                    |
| Yekaterinburgh |   310,255.90         | 7.62        | 99     | 3,133.90                | 4                 | 77,563.97                    |
| Perm           |   260,324.20         | 6.40        | 48     | 5,423.42                | 5                 | 52,064.84                    |
| Omsk           |   142,487.70         | 3.50        | 64     | 2,226.37                | 5                 | 28,497.54                    |
| Kirov          |   124,397.60         | 3.06        | 23     | 5,408.59                | 1                 | 124,397.60                   |
| Murmansk       |   124,074.20         | 3.05        | 20     | 6,203.71                | 3                 | 41,358.07                    |
| Voronezh       |   112,347.30         | 2.76        | 63     | 1,783.29                | 4                 | 28,086.82                    |
| Cherepovets    |   108,075.00         | 2.66        | 31     | 3,486.29                | 1                 | 108,075.00                   |
| Surgut         |    80,063.10         | 1.97        | 10     | 8,006.31                | 2                 | 40,031.55                    |
| Khabarovsk     |    78,917.50         | 1.94        | 36     | 2,192.15                | 6                 | 13,152.92                    |

#### Key Takeaways

- **Moscow alone accounts for over 45%** of the total revenue in the dataset.
- Together, these top 10 cities account for **79.12%** of total revenue.
- This strong regional skew suggests that operations, logistics, and customer acquisition strategies are centered in or around major metropolitan areas.
- The remaining **20.88% of revenue is distributed across 70+ smaller cities**. This demonstrates a healthy level of market reach and brand presence nationwide. It indicates potential for further penetration in secondary markets, where growth might be more cost-effective than in already saturated major cities.

#### Individual Region Insights

- **Moscow** stands as a diversified and reliable revenue hub, with **62 unique customers** and over **1,400 orders**. Its **moderate average order value** shows depth in client size, not just a few major buyers.

- **Yekaterinburgh** and **Perm** both generate strong revenue with just a **handful of clients**, which presents an opportunity to **nurture and retain high-value accounts**.

- **Surgut** showcases a striking **$8,000+ average order value**, showing potential to expand sales from a very small base of customers.

- **Kirov** and **Cherepovets** each rely on **one high-value client**. While lucrative, this introduces risk — these cities require careful monitoring or efforts to **diversify the client base**.

> 📌 **Insight**: While Moscow anchors the business, regional cities like Yekaterinburgh, Perm, and Surgut offer opportunities to grow key accounts. The presence of single-client cities generating six-figure revenue shows that strategic expansion and retention plans in these areas could yield high returns with minimal operational overhead.


In [0]:
%sql
-- 🌍 Top 10 Revenue-Generating Regions with Advanced Metrics
SELECT 
  c.Region,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue,
  ROUND(
    100 * SUM(f.QuantitySold * f.UnitSalesPrice) / 
    SUM(SUM(f.QuantitySold * f.UnitSalesPrice)) OVER (), 
    2
  ) AS RevenuePercentage,
  COUNT(DISTINCT f.OrderID) AS TotalOrders,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice) / COUNT(DISTINCT f.OrderID), 2) AS AvgOrderValue,
  COUNT(DISTINCT f.CustomerID) AS UniqueCustomers,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice) / COUNT(DISTINCT f.CustomerID), 2) AS AvgRevenuePerCustomer

FROM fact_sales f
JOIN dim_customers c ON f.CustomerID = c.CustomerID
GROUP BY c.Region
ORDER BY TotalRevenue DESC
LIMIT 10;

Region,TotalRevenue,RevenuePercentage,TotalOrders,AvgOrderValue,UniqueCustomers,AvgRevenuePerCustomer
Moscow,1878887.57,46.16,1412,1330.66,62,30304.64
Yekaterinburgh,310255.9,7.62,99,3133.9,4,77563.97
Perm,260324.2,6.4,48,5423.42,5,52064.84
Omsk,142487.7,3.5,64,2226.37,5,28497.54
Kirov,124397.6,3.06,23,5408.59,1,124397.6
Murmansk,124074.2,3.05,20,6203.71,3,41358.07
Voronezh,112347.3,2.76,63,1783.29,4,28086.82
Cherepovets,108075.0,2.66,31,3486.29,1,108075.0
Surgut,80063.1,1.97,10,8006.31,2,40031.55
Khabarovsk,78917.5,1.94,36,2192.15,6,13152.92


###5.5 - 🔍 Revenue by Lead Source

This analysis helps understand where the company’s customers are coming from and which acquisition channels are driving the most revenue.

| Lead Source                                        | Total Revenue (USD) | % of Total Revenue |
|----------------------------------------------------|--------------------|---------------------|
| Referral by the Central Office                     | 2,482,829.15       | 61.00%              |
| Advertisement in National Wholesale Magazine       |   784,164.80       | 19.27%              |
| Sales Calls or Visits                              |   549,378.30       | 13.50%              |
| Other                                              |   238,518.32       | 5.86%               |
| *No information (null)*                            |     7,234.30       | 0.18%               |
| Organic Search                                     |     3,497.70       | 0.09%               |
| Trade Shows                                        |     3,340.60       | 0.08%               |
| Referral by Third Parties                          |       953.20       | 0.02%               |
| Advertisement in Regional Wholesaler Magazine      |       140.30       | 0.00%               |

- The **vast majority of revenue (61%) comes from referrals by the Central Office**, suggesting a strong centralized B2B acquisition model.
- **Magazine advertising and direct sales** efforts (calls and visits) also show strong returns, justifying continued investment in traditional B2B outreach.
- However, **Organic Search accounts for only 0.09%** of revenue. Given the business’s scale, this indicates low brand awareness or poor digital presence.
- Trade show performance is also negligible, suggesting either low participation or low conversion.
- The presence of null values in ~0.18% of the data could be resolved to improve lead source tracking and attribution, but is not a priority due to its small percentage (this could possibly be fixed by identifying who made the sale and manually inputing the missing data).

> 📌 **Insight**: The company has a highly effective centralized acquisition engine but is **missing opportunities in digital channels**. Building SEO, content marketing, or online visibility could open additional acquisition streams and reduce reliance on traditional methods.

In [0]:
%sql
-- 📣 Revenue by Lead Source
SELECT 
  c.LeadSource,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue,
  ROUND(
    100 * SUM(f.QuantitySold * f.UnitSalesPrice) / 
    SUM(SUM(f.QuantitySold * f.UnitSalesPrice)) OVER (), 
    2
  ) AS RevenuePercentage
FROM fact_sales f
JOIN dim_customers c ON f.CustomerID = c.CustomerID
GROUP BY c.LeadSource
ORDER BY TotalRevenue DESC;

LeadSource,TotalRevenue,RevenuePercentage
Referral by the Central Office,2482829.15,61.0
Advertisement in National Wholesale Magazine,784164.8,19.27
Sales Calls or Visits,549378.3,13.5
Other,238518.32,5.86
null,7234.3,0.18
Organic Search,3497.7,0.09
Trade Shows,3340.6,0.08
Referral by Third Parties,953.2,0.02
Advertisement in Regional Wholesaler Magazine,140.3,0.0


###5.6 - 🧩 Revenue by Customer Class

This analysis confirms the company’s strong focus on B2B operations, with nearly all revenue originating from wholesalers.

| Customer Class               | Total Revenue (USD) | % of Total Revenue |
|------------------------------|--------------------|---------------------|
| Large-Scale Wholesaler-1     | 2,523,498.00       | 62.00%              |
| Small-Scale Wholesaler       | 1,111,037.80       | 27.30%              |
| HyPermarket                  |   222,615.10       | 5.47%               |
| Large-Scale Wholesaler-2     |   168,490.75       | 4.14%               |
| Retailer                     |    39,295.30       | 0.97%               |
| Consumer                     |     4,021.42       | 0.10%               |
| Branch                       |     1,098.30       | 0.03%               |

- Around **93% of all revenue comes from wholesale clients**, with minimal presence in retail or direct-to-consumer channels.
- This reinforces the brand’s positioning as a **wholesaler-focused operation**, built on volume sales and business-to-business relationships.
- The company could benefit from **segmenting large-scale wholesalers more precisely** (e.g., by size, location, or sales volume) to identify key accounts that deserve custom pricing, account management, or relationship-building.
- The **HyPermarket segment** presents an opportunity to develop a tailored offering or partnership strategy given its standalone contribution.
- Direct consumer expansion is **not currently advised**, as it could conflict with established B2B trust and distribution dynamics.

> 📌 **Insight**: Focused relationship management and tiered service strategies for top wholesalers could improve retention and lifetime value. Customizing outreach for top accounts (especially HyPermarkets) may open up new growth opportunities within the same channel structure.

In [0]:
%sql
-- 🧩 Revenue by Customer Class
SELECT 
  c.CustomerClass,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue,
  ROUND(
    100 * SUM(f.QuantitySold * f.UnitSalesPrice) / 
    SUM(SUM(f.QuantitySold * f.UnitSalesPrice)) OVER (), 
    2
  ) AS RevenuePercentage
FROM fact_sales f
JOIN dim_customers c ON f.CustomerID = c.CustomerID
GROUP BY c.CustomerClass
ORDER BY TotalRevenue DESC;

CustomerClass,TotalRevenue,RevenuePercentage
Large-Scale Wholesaler-1,2523498.0,62.0
Small-Scale Wholesaler,1111037.8,27.3
HyPermarket,222615.1,5.47
Large-Scale Wholesaler-2,168490.75,4.14
Retailer,39295.3,0.97
Consumer,4021.42,0.1
Branch,1098.3,0.03


###5.7 - 🔁 Revenue by Customer Status (Active vs. Discontinued)

This metric shows the share of revenue generated by currently active customers versus those marked as discontinued.

| Customer Status | Total Revenue (USD) | % of Total Revenue |
|------------------|--------------------|---------------------|
| Active           | 3,933,077.75       | 96.63%              |
| Discontinued     |   136,978.92       | 3.37%               |

- **Churn appears to be low**, with over 96% of revenue coming from active customers.
- Discontinued accounts contributed only a small portion of past revenue, indicating strong ongoing customer relationships and account stability.
- However, even a 3.37% churn rate could be meaningful depending on the **size or value of individual clients**.
- These inactive clients may still represent **low-effort recovery opportunities**. Re-engagement campaigns, surveys, or personalized promotions could help understand their disengagement and potentially recover lost revenue or even prevent other active customers from churning.

In [0]:
%sql
-- 🔁 Revenue by Customer Status (Active vs. Discontinued)
SELECT 
  CASE 
    WHEN c.Discontinued = 1 THEN 'Discontinued'
    WHEN c.Discontinued = 0 THEN 'Active'
    ELSE 'Unknown'
  END AS CustomerStatus,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue,
  ROUND(
    100 * SUM(f.QuantitySold * f.UnitSalesPrice) / 
    SUM(SUM(f.QuantitySold * f.UnitSalesPrice)) OVER (),
    2
  ) AS RevenuePercentage
FROM fact_sales f
JOIN dim_customers c ON f.CustomerID = c.CustomerID
GROUP BY CustomerStatus
ORDER BY TotalRevenue DESC;

CustomerStatus,TotalRevenue,RevenuePercentage
Active,3933077.75,96.63
Discontinued,136978.92,3.37


###5.8 - 🏆 Top 10 Customers – Performance Deep Dive

This table highlights the company's 10 most valuable customers, combining not only total revenue but also average order value, number of unique product categories purchased, and customer engagement levels. These metrics offer a comprehensive view of each client’s contribution to the business and help identify where to focus relationship-building, cross-selling strategies, and account development efforts for maximum ROI.

| Customer Name | Total Revenue (USD) | Revenue % | Total Orders | Avg. Order Value (USD) | Unique Categories |
|---------------|---------------------|-----------|---------------|--------------------------|--------------------|
| C5            | 570,217.80          | 14.01%    | 130           | 4,386.29                | 5                  |
| C2            | 384,881.90          | 9.46%     | 126           | 3,054.62                | 8                  |
| C13           | 225,268.60          | 5.53%     | 37            | 6,088.34                | 5                  |
| C209          | 222,615.10          | 5.47%     | 39            | 5,708.08                | 5                  |
| C75           | 157,444.00          | 3.87%     | 46            | 3,422.70                | 7                  |
| C24           | 134,352.90          | 3.30%     | 36            | 3,732.02                | 6                  |
| C56           | 124,397.60          | 3.06%     | 23            | 5,408.59                | 7                  |
| C40           | 117,903.10          | 2.90%     | 17            | 6,935.48                | 8                  |
| C6            | 109,633.10          | 2.69%     | 50            | 2,192.66                | 3                  |
| C136          | 108,075.00          | 2.66%     | 31            | 3,486.29                | 6                  |

---

### Key Insights:

- The top 10 customers together account for **54% of total revenue**, with the top 2 alone contributing more than **23%**.
- While these accounts clearly deserve special attention (e.g. strategic pricing, dedicated reps), the business still serves **over 200 clients**, who collectively make up nearly **46% of the revenue**.
- This **balanced structure** indicates strong client diversification and reduced dependency on any single account.

---

### Customer Specific Inisghts:

- **C5 and C2** stand out as **top revenue generators**, combining high order volumes with strong consistency, making them high-priority clients.
- Customers like **C13**, **C209**, **C56**, and **C40** have **exceptional average order values** (above USD $5,000), suggesting strong potential for account-based marketing or VIP treatment.
- **C40** in particular manages to purchase across 8 product categories with just 17 orders — a sign of high-value, intentional procurement.
- **C6** shows steady engagement through frequent purchases, albeit with lower value per order — a candidate for volume discounts or product bundling strategies.
- Clients engaging with a wide variety of product categories (like **C2**, **C75**, and **C40**) are ideal targets for **cross-selling** and product experimentation.

> 📌 **Insight**: The company should maintain high-touch strategies for top accounts, while preserving strong service and communication across the full client base. This mix of depth and breadth reflects a resilient and well-managed B2B operation.

In [0]:
%sql
-- 🧑‍💼 Customer Performance Deep Dive
-- Combines revenue, order volume, AOV, and category diversity per customer

SELECT 
  c.CustomerName,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice) / (SELECT SUM(QuantitySold * UnitSalesPrice) FROM fact_sales) * 100, 2) AS RevenuePercentage,
  COUNT(DISTINCT f.OrderID) AS TotalOrders,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice) / COUNT(DISTINCT f.OrderID), 2) AS AvgOrderValue,
  COUNT(DISTINCT p.Category) AS UniqueCategories

FROM fact_sales f
JOIN dim_customers c ON f.CustomerID = c.CustomerID
JOIN dim_products p ON f.ProductID = p.ProductID

GROUP BY c.CustomerName
ORDER BY TotalRevenue DESC
LIMIT 10;

CustomerName,TotalRevenue,RevenuePercentage,TotalOrders,AvgOrderValue,UniqueCategories
C5,570217.8,14.01,130,4386.29,5
C2,384881.9,9.46,126,3054.62,8
C13,225268.6,5.53,37,6088.34,5
C209,222615.1,5.47,39,5708.08,5
C75,157444.0,3.87,46,3422.7,7
C24,134352.9,3.3,36,3732.03,6
C56,124397.6,3.06,23,5408.59,7
C40,117903.1,2.9,17,6935.48,8
C6,109633.1,2.69,50,2192.66,3
C136,108075.0,2.66,31,3486.29,6


###5.9 - 🧑‍💼 Employee Performance Analysis

This comprehensive view summarizes each employee’s contribution across multiple dimensions: total revenue, revenue share, number of orders, average order value (AOV), and unique customers handled.

> **Note:** The original employee dataset includes 15 employees, but only 10 appear in this analysis. This suggests that **5 employees either haven’t logged sales** or are engaged in **non-sales roles** (e.g., logistics, purchasing, operations). This helps narrow focus to only active sales performers.

---

#### 📊 Employee Sales Performance Table

| EmployeeName | TotalRevenue (USD) | RevenuePercentage (%) | TotalOrders | RevenuePerOrder (USD) | UniqueCustomers |
|--------------|--------------------|------------------------|-------------|------------------------|------------------|
| E5           | 1,914,747.70       | 47.04                  | 1,056       | 1,813.21               | 153              |
| E7           | 569,722.80         | 14.00                  | 228         | 2,498.78               | 77               |
| E2           | 551,670.37         | 13.55                  | 387         | 1,425.50               | 56               |
| E1           | 490,210.50         | 12.04                  | 293         | 1,673.07               | 83               |
| E13          | 333,873.00         | 8.20                   | 172         | 1,941.12               | 57               |
| E11          | 100,734.70         | 2.48                   | 62          | 1,624.75               | 34               |
| E6           | 32,340.20          | 0.79                   | 26          | 1,243.85               | 17               |
| E14          | 31,787.60          | 0.78                   | 19          | 1,673.03               | 15               |
| E15          | 28,778.30          | 0.71                   | 11          | 2,616.21               | 7                |
| E12          | 16,191.50          | 0.40                   | 7           | 2,313.07               | 6                |

---

#### 🔍 Key Highlights:

- **Employee 5** is the company’s top performer by a wide margin, accounting for **47.04% of total revenue (USD 1,914,747.70)**, the **highest number of orders (1,056)**, and the **largest customer base (153 clients)**. While their AOV isn't the highest, their scale makes them a strategic asset. This employee could **mentor others** or lead internal workshops to share best practices.

- **Employee 7** ranks second in total revenue but stands out with a **very high AOV of USD 2,498.78**. This suggests specialization in **larger or more premium deals**, which could inform strategies for targeting high-value segments.

- **Employee 2** handles more orders than Employee 7 but has a lower AOV (USD 1,425.50). With fewer unique clients, they show signs of **strong repeat business** — suggesting room to **expand client base** while maintaining retention.

- **Employees 1 and 13** show **balanced and consistent performance** across all metrics. While their total revenue is lower than the top 3, their **solid AOV and customer reach** demonstrate they are dependable, scalable contributors.

- **Bottom 5 employees (11, 6, 14, 15, 12)** have lower overall impact. They likely need more support, have **limited client assignments**, or are **new to the sales role**. While their volume is low, several show healthy AOVs.

- **Employees 15 and 12**, despite ranking lower in total orders and customer count, stand out with **exceptionally high AOVs** (USD 2,616.21 and USD 2,313.07 respectively). This could imply they are managing **select premium clients** or are **high-potential newcomers**.

---

#### 💡 Strategic Insight:

This table allows management to segment sales employees based on performance and potential:

- **Volume-driven sellers** (like E5 and E2) can be supported with tools to scale more efficiently.
- **High-AOV employees** (like E7, E15, and E12) can be focused on **key accounts or premium offerings**.
- **Underutilized employees** may need **training, reassignment, or mentoring**.
- **Balanced performers** like E1 and E13 are ideal for **long-term scalable sales growth**.

This multi-metric analysis provides a strong base for **data-driven talent management and sales enablement strategies**.

---

In [0]:
%sql
-- 🧑‍💼 Wide Employee Dimension View:
-- This query combines revenue performance, customer reach, and order efficiency 
-- for each employee, providing a complete picture of their impact on sales.

SELECT 
  e.EmployeeName,
  
  -- 📦 Revenue
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue,
  
  -- 📊 Revenue share across all employees
  ROUND(
    100 * SUM(f.QuantitySold * f.UnitSalesPrice) / 
    SUM(SUM(f.QuantitySold * f.UnitSalesPrice)) OVER (), 
    2
  ) AS RevenuePercentage,

  -- 🧾 Total unique orders handled
  COUNT(DISTINCT f.OrderID) AS TotalOrders,

  -- 💰 Average Order Value (AOV)
  ROUND(
    SUM(f.QuantitySold * f.UnitSalesPrice) / COUNT(DISTINCT f.OrderID),
    2
  ) AS RevenuePerOrder,

  -- 🤝 Number of different customers served
  COUNT(DISTINCT f.CustomerID) AS UniqueCustomers

FROM fact_sales f
JOIN dim_employees e ON f.EmployeeID = e.EmployeeID
GROUP BY e.EmployeeName
ORDER BY RevenuePercentage DESC;

EmployeeName,TotalRevenue,RevenuePercentage,TotalOrders,RevenuePerOrder,UniqueCustomers
  E 5  ,1914747.7,47.04,1056,1813.21,153
  E 7  ,569722.8,14.0,228,2498.78,77
  E 2  ,551670.37,13.55,387,1425.5,56
  E 1  ,490210.5,12.04,293,1673.07,83
  E 1 3  ,333873.0,8.2,172,1941.12,57
  E 1 1  ,100734.7,2.48,62,1624.75,34
  E 6  ,32340.2,0.79,26,1243.85,17
  E 1 4  ,31787.6,0.78,19,1673.03,15
  E 1 5  ,28778.3,0.71,11,2616.21,7
  E 1 2  ,16191.5,0.4,7,2313.07,6


###5.10 - 📆 Revenue Trend Over Time

- The data covers a 3-year period (from mid-2003 to mid-2006).
- While there’s no strong seasonal pattern, there’s a **general upward trajectory**, with **2006 showing higher revenue months**, peaking at over R$212K in April.
- Order volume and revenue tend to rise together, indicating that **order frequency** is the key driver of growth (vs. higher average order value).
- The business seems to be **stabilizing and growing modestly over time** but there are no major patterns in the time series.

> 📌 **Note**: The trend is not very pronounced, so the real value may lie in exploring customer behavior, product strategies, or regional performance rather than time-based forecasting.

In [0]:
%sql
-- 📈 Monthly Revenue Trend
SELECT 
  DATE_FORMAT(f.OrderDateKey, 'yyyy-MM') AS Month,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue,
  COUNT(DISTINCT f.OrderID) AS TotalOrders
FROM fact_sales f
GROUP BY DATE_FORMAT(f.OrderDateKey, 'yyyy-MM')
ORDER BY Month;

Month,TotalRevenue,TotalOrders
2003-07,55627.3,36
2003-08,114025.6,72
2003-09,52547.75,49
2003-10,91976.7,78
2003-11,60076.22,65
2003-12,76363.6,75
2004-01,90237.5,70
2004-02,127283.5,84
2004-03,131775.5,77
2004-04,184751.1,63


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

###5.11 - 💰 Average Order Value (AOV) by Customer Class

This analysis highlights how much each customer class spends per order on average, helping identify which segments offer higher ticket sizes and possibly higher margins.

| Customer Class             | Avg Order Value (USD) | Total Orders | Total Revenue (USD) |
|----------------------------|----------------------|--------------|---------------------|
| HyPermarket                | 5,708.08             | 39           | 222,615.10          |
| Large-Scale Wholesaler-1   | 2,466.76             | 1,023        | 2,523,498.00        |
| Small-Scale Wholesaler     | 2,334.11             | 476          | 1,111,037.80        |
| Large-Scale Wholesaler-2   | 423.34               | 398          | 168,490.75          |
| Retailer                   | 280.68               | 140          | 39,295.30           |
| Branch                     | 274.58               | 4            | 1,098.30            |
| Consumer                   | 22.22                | 181          | 4,021.42            |

- **HyPermarkets stand out as the highest-value buyers per transaction**, more than **2x the AOV of major wholesalers**. This validates earlier insights that this segment is worth targeting with tailored offers or strategic partnerships.
- **Large-Scale Wholesaler-1** clients place high-value and high-volume orders, forming the core of the business.
- AOV drops off significantly in **Wholesaler-2**, Retailer, and Consumer classes — reinforcing that these are less profitable per transaction.
- The **Consumer class has the lowest AOV by far (22.22USD)**, further supporting the decision to **avoid retail expansion** and focus on the more lucrative B2B market.

> 📌 **Insight**: HyPermarkets are both high-value and low-frequency — ideal for strategic account development. Meanwhile, optimizing pricing or order incentives for Wholesaler-2 and Retailers could help improve margins in those segments.

In [0]:
%sql
-- 💰 Average Order Value by Customer Class
SELECT 
  c.CustomerClass,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice) / COUNT(DISTINCT f.OrderID), 2) AS AvgOrderValue,
  COUNT(DISTINCT f.OrderID) AS TotalOrders,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue
FROM fact_sales f
JOIN dim_customers c ON f.CustomerID = c.CustomerID
GROUP BY c.CustomerClass
ORDER BY AvgOrderValue DESC;

CustomerClass,AvgOrderValue,TotalOrders,TotalRevenue
HyPermarket,5708.08,39,222615.1
Large-Scale Wholesaler-1,2466.76,1023,2523498.0
Small-Scale Wholesaler,2334.11,476,1111037.8
Large-Scale Wholesaler-2,423.34,398,168490.75
Retailer,280.68,140,39295.3
Branch,274.58,4,1098.3
Consumer,22.22,181,4021.42


###5.12 - 🔁 Revenue by Customer Loyalty

This analysis compares how much revenue comes from repeat customers versus one-time buyers.

| Customer Type     | Total Revenue (USD) | % of Total Revenue |
|-------------------|--------------------|---------------------|
| Repeat Customer   | 4,013,391.57       | 98.61%              |
| One-Time Customer |    56,665.10       | 1.39%               |

- The business is overwhelmingly driven by **repeat customers**, which is expected for a B2B wholesaler operation.
- This reinforces earlier insights that **customer retention is strong**, and that most clients place multiple orders over time.
- While there's little value in focusing on one-timers, **the low churn and repeat pattern show high satisfaction and loyalty** among clients.

> 📌 **Insight**: A retention-driven sales model is already in place — meaning efforts can shift toward optimizing top-tier client relationships, not just acquiring new customers.

In [0]:
%sql
-- 🔁 Repeat vs. One-Time Buyers
WITH customer_order_counts AS (
  SELECT 
    CustomerID,
    COUNT(DISTINCT OrderID) AS TotalOrders
  FROM fact_sales
  GROUP BY CustomerID
)

SELECT 
  CASE 
    WHEN c.TotalOrders > 1 THEN 'Repeat Customer'
    ELSE 'One-Time Customer'
  END AS CustomerType,
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue,
  ROUND(
    100 * SUM(f.QuantitySold * f.UnitSalesPrice) / 
    SUM(SUM(f.QuantitySold * f.UnitSalesPrice)) OVER (), 
    2
  ) AS RevenuePercentage
FROM fact_sales f
JOIN customer_order_counts c ON f.CustomerID = c.CustomerID
GROUP BY CustomerType
ORDER BY TotalRevenue DESC;

CustomerType,TotalRevenue,RevenuePercentage
Repeat Customer,4013391.57,98.61
One-Time Customer,56665.1,1.39


###5.13 - 🚚 Shipping Method Overview

| Shipping Method | Total Orders Shipped | Total Revenue (USD) | Avg Order Value (USD) | Total Freight Charges (USD) | Avg Freight/Order (USD) |
|------------------|----------------------|----------------------|------------------------|------------------------------|--------------------------|
| Ex Works         | 106,392              | 4,069,603.07         | 38.25                  | 1,373,294.28                 | 12.91                    |
| Container        | 31                   | 453.60               | 14.63                  | 0.00                         | 0.00                     |

Despite being part of the dataset, the **Shipping Method field lacks granularity** and strategic diversity. Over **99% of orders** were processed using the `Ex Works` method, making it the only relevant entry for analytical purposes.

While freight charges are present and can be analyzed, the **shipping method data itself is not rich or varied enough** to drive operational or strategic insights. This may indicate:
- A consistent logistical policy across the business, or
- A **limitation in data entry or tracking** practices related to this field.

> 📌 **Insight**: The company may benefit from improving the granularity and consistency of its shipping method data to better support future logistics analysis or optimization projects.

In [0]:
%sql
-- 📦 Shipping Method Performance Overview
SELECT 
  s.ShippingMethod,                                      -- Name of shipping method
  COUNT(*) AS TotalOrdersShipped,                        -- Total number of orders using this method
  ROUND(SUM(f.QuantitySold * f.UnitSalesPrice), 2) AS TotalRevenue,  -- Revenue from these orders
  ROUND(AVG(f.QuantitySold * f.UnitSalesPrice), 2) AS AvgOrderValue, -- Average revenue per order
  ROUND(SUM(f.FreightCharge), 2) AS TotalFreightCharges,             -- Total freight costs
  ROUND(AVG(f.FreightCharge), 2) AS AvgFreightChargePerOrder         -- Average freight per order
FROM fact_sales f
JOIN dim_shipping_methods s ON f.ShippingMethodID = s.ShippingMethodID
GROUP BY s.ShippingMethod
ORDER BY TotalOrdersShipped DESC;

ShippingMethod,TotalOrdersShipped,TotalRevenue,AvgOrderValue,TotalFreightCharges,AvgFreightChargePerOrder
Ex Works,106392,4069603.07,38.25,1373294.28,12.91
Container,31,453.6,14.63,0.0,0.0


##6. ✅ Final Considerations

This project allowed for a deep exploration of the company’s sales data by building a star schema and conducting targeted business analysis using PySpark and SQL within the Databricks environment. Despite certain limitations in the raw data, the structure and strategy applied throughout the project delivered valuable insights across multiple dimensions of the business — from customers and products to employees and regional reach.

---

### 🏢 Business Takeaways

The analysis clearly paints the picture of an established B2B operation with strong specialization, market presence, and customer diversity. Below are the key insights and patterns discovered:

- **Category Specialization**:  
  Over **85% of sales revenue** comes from briefs, with **78% coming specifically from women's panties**. This intense focus confirms a clear specialization in the female intimate apparel segment. However, the product portfolio includes items like men's briefs, undershirts, and even socks. While these generate relatively low revenue, they contribute to **brand depth**, positioning the company as an expert in full-scope intimate wear. These items enhance the brand's identity and emotional appeal — as long as their production aligns with actual demand, they provide strategic value beyond revenue.

- **Customer Base**:  
  The company serves **over 220 clients**, and while the **top 10 customers account for 54% of the revenue**, the remaining revenue is distributed across many smaller clients — showing strong **client diversification** and reducing dependency.  
  Customers such as **C13, C209, C56, and C40** stood out for their **high average order value**, even if they placed fewer orders — suggesting they should be prioritized for relationship development or personalized account strategies.

- **Customer Segmentation**:  
  The company is almost entirely **wholesale-focused**, with minimal consumer presence. Segments such as **hypermarkets** and **large-scale wholesalers** drive the majority of the business. Better naming conventions and structured classifications (e.g., identifying hypermarkets or branches more clearly) would unlock stronger CRM strategies.

- **Lead Source & Customer Acquisition**:  
  Over **60% of revenue** comes from customers referred by the **central office**, and almost **20% from traditional advertising**. However, **organic search barely registers**, indicating a huge opportunity for **digital visibility and inbound marketing** to support brand growth.

- **Geographic Reach**:  
  The company has **strong presence in major metropolitan areas**, especially **Moscow**, which accounts for **46% of revenue**. Yet, it also sells to **70+ other cities**, and about **21% of the revenue comes from these smaller markets** — showing strong distribution and potential for regional expansion through cost-effective sales strategies.

- **Employee Performance**:  
  Employee E5 was identified as the company's top performer in both revenue and unique clients served. The bottom five employees handled fewer clients and orders, but some — like E15 and E12 — had **impressive revenue per order**, possibly indicating **recent hires or high-potential reps**. These insights can fuel sales coaching, compensation plans, and growth opportunities.

- **Customer Behavior**:  
  Nearly **99% of the revenue** comes from **repeat customers**, confirming strong retention and satisfaction. This suggests the business has a reliable and recurring demand base and reinforces its strength as a **B2B player**.

- **Product Data**:  
  While many products had generic or coded names (e.g., “B105”) and lacked useful attributes like color or size, there’s a clear opportunity to improve catalog quality for better **product performance analysis** and **trend tracking**.

- **Revenue vs. Payment Discrepancy**:  
  The project found that only **648 of 2,261 orders** had corresponding payment information. Because of this, the analysis was based on **order revenue** rather than **recorded payments**, which likely reflects more consistent and accurate business activity.

---

### 🔍 On Data Quality & Opportunities for Improvement

Although key insights were obtained, several **data quality issues** were encountered:

- **Missing Payment Info**: The payments dataset lacked complete coverage and contained inconsistencies in payment methods, making it unreliable for financial tracking.
- **Shipping Method Granularity**: With only two available methods and minimal variation, shipping insights were limited. More detailed shipping records would allow optimization of logistics and cost management.
- **Product Attributes**: The lack of color, consistent sizes, and descriptive product names made deeper product analysis difficult. A stronger product catalog would enhance reporting, personalization, and marketing strategies.
- **Customer Classes & Identifiers**: Certain segments (e.g., Large-Scale Wholesaler, Large-Scale Wholesaler-1, Large-Scale Wholesaler-2) were not labeled clearly enough to support specific campaigns or outreach.
- **Schema Suggestions**: Some datasets (e.g., inventory transactions, suppliers) were not used in the final fact table, but could support future **star schemas** for:
  - **Inventory Management**
  - **Supplier Performance**
  - **Procurement Analysis**
  - **Product Lifecycle Tracking**

---

### 🧪 Tools, Technologies, and Methodology

This project was developed entirely within **Databricks Community Edition** using **Apache Spark**, **SQL**, and **Python (PySpark)**.

- **Apache Spark** enabled **distributed data processing**, **schema inference**, and powerful **lazy evaluation**, making it ideal for large-scale joins, transformations, and analytical queries.
- **Databricks Notebooks** allowed for an organized, version-controlled development environment, combining **code, output, and markdown-based storytelling** in a single interface.
- **SQL Integration** was essential for executing complex queries over the star schema once the tables were registered as temporary views, providing **business-level insights**.
- The project also involved detailed **data cleaning**, **schema normalization**, **foreign key tracing**, and **star schema modeling**, all foundational steps for building a scalable data warehouse.

---

### 🚀 Future Work

Several next steps could continue to grow the value of this project:

- **Develop additional star schemas** to support inventory, procurement, or marketing analytics.
- **Integrate external benchmarking data** to compare pricing, performance, or product popularity with competitors.
- **Implement dashboards or APIs** (e.g., Streamlit, Power BI) for interactive stakeholder access to insights.
- **Automate pipelines** using tools like **Airflow**, **Delta Live Tables**, or **Make.com** for continuous data updates and real-time reporting.
- Explore **machine learning models** for churn prediction, customer segmentation, or demand forecasting.

---

This project successfully brought structure to raw, fragmented data and turned it into actionable insights. The combination of data modeling, business logic, and analytical storytelling demonstrates how Spark-powered analytics can drive smarter decisions and deeper understanding across any retail or B2B-focused organization.